In [1]:
cd models/research/object_detection

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research/object_detection


In [3]:
import numpy as np
import math
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from PIL import Image
import pytesseract

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from utils import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.layers import *

from keras.applications import MobileNetV2
from keras.applications import InceptionResNetV2

from keras.models import Model
from keras.models import model_from_json


ImportError: No module named 'utils'

In [4]:
target_size = [480, 480]
target_w = 480 # target sizes of image in model input
target_h = 480 #target sizes of image in model input

grid_size = [15, 15]
grid_y_axis = 15  # each image is to be segmented to 13 x 13 grid
grid_x_axis = 15  # # each image is to be segmented to 13 x 13 grid

grid_w = target_w / grid_x_axis  # grid cell width
grid_h = target_h / grid_y_axis  # grid cell height

channels = 3
num_anchors = 3
class_num = 5 # vendor, invoice, inv_date, po, buyer
info = 5 + class_num    # pc, x, y, h, w, and class probabilities

categories = ['vendor', 'invoice', 'inv_date', 'po', 'buyer'] # details of classes

In [5]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/8-6-new-scan'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_new_image = ['/home/scar3crow/Downloads/8-6-new-scan/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_new_image.sort() # Sorting the list

num_images = len(inv_new_image)

print('Number of images = ', num_images)
inv_new_image[24]

Number of images =  36


'/home/scar3crow/Downloads/8-6-new-scan/50a.jpg'

In [6]:
# Check sizes of exiting images & Create a Dataframe with image id and height(row) and width(column):

rows = []
columns = []
image_sl = []
df_new = pd.DataFrame()

for i in range(len(inv_new_image)):
    image = cv2.imread(inv_new_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_new_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df_new.insert(loc=0, column='image_serial', value=image_num)
df_new.insert(loc=1, column='rows', value=row_values)
df_new.insert(loc=2, column='columns', value=col_values)

df_new.head(3)

,image_serial,rows,columns
0,/home/scar3crow/Downloads/8-6-new-scan/101a.jpg,160,416
1,/home/scar3crow/Downloads/8-6-new-scan/102a.jpg,406,870
2,/home/scar3crow/Downloads/8-6-new-scan/103a.jpg,260,416


In [7]:
# Loading output of VGG Image Annotation tool and create a dataframe

r_new_data = pd.read_csv('/home/scar3crow/Downloads/via_new_data.csv')
num_obj = r_new_data['region_count'][0] # number of objects in each photo
r_new_data.drop(r_new_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns
r_new_data.sort_values(by=['#filename'], ascending=True) # Sorting based on image-id
num_images = r_new_data["#filename"].nunique() # Find out number of unique images

print('Number of classes = ', num_obj)
print('Number of unique images = ', num_images)
r_new_data[58:61]

Number of classes =  5
Number of unique images =  36


,#filename,region_shape_attributes,region_attributes
58,63a.jpg,"{""name"":""rect"",""x"":211,""y"":64,""width"":76,""heig...","{""text"":""po""}"
59,63a.jpg,"{""name"":""rect"",""x"":2,""y"":68,""width"":165,""heigh...","{""text"":""buyer""}"
60,101a.jpg,"{""name"":""rect"",""x"":6,""y"":23,""width"":119,""heigh...","{""text"":""vendor""}"


In [8]:
# Making a dataframe for Image_id, x, y, width, height, class, image_width and image_height

x = []
y = []
width = []
height = []
obj_class = []
i_width = []
i_height = []
img_path = []
img_index = []

for i in range(len(r_new_data)):
    
    r_size = r_new_data.values[i, 1][1:(len(r_new_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_new_data.values[i, 2][1:(len(r_new_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
    foto_id = r_new_data['#filename'][i]
    i_path = '/home/scar3crow/Downloads/8-6-new-scan/' + foto_id
    foto_index = int(df_new[df_new['image_serial'] == i_path].index[0])
    foto_width = df_new['columns'][foto_index]
    foto_height = df_new['rows'][foto_index]
    i_width.append(foto_width)
    i_height.append(foto_height)
    img_path.append(i_path)
    img_index.append(foto_index)
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)
i_width_values = pd.Series(i_width)
i_height_values = pd.Series(i_height)
img_path_values = pd.Series(img_path)
img_index_values = pd.Series(img_index)

r_new_data.insert(loc=1, column='img_idx', value=img_index_values)
r_new_data.insert(loc=2, column='i_path', value=img_path_values)
r_new_data.insert(loc=3, column='x', value=x_values)
r_new_data.insert(loc=4, column='y', value=y_values)
r_new_data.insert(loc=5, column='width', value=width_values)
r_new_data.insert(loc=6, column='height', value=height_values)
r_new_data.insert(loc=7, column='obj_class', value=class_values)
r_new_data.insert(loc=8, column='img_wd', value=i_width_values)
r_new_data.insert(loc=9, column='img_ht', value=i_height_values)

r_new_data.drop(r_new_data.columns[[10, 11]], axis=1, inplace=True) # reduce unnecessary columns

r_new_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_new_data[3:6]


,img_id,img_idx,i_path,x,y,width,height,obj_class,img_wd,img_ht
3,50a.jpg,24,/home/scar3crow/Downloads/8-6-new-scan/50a.jpg,221,59,103,24,po,416,209
4,50a.jpg,24,/home/scar3crow/Downloads/8-6-new-scan/50a.jpg,5,57,206,56,buyer,416,209
5,51a.jpg,25,/home/scar3crow/Downloads/8-6-new-scan/51a.jpg,5,0,120,56,vendor,416,194


In [9]:
print('Number of unique images = ', r_new_data['img_id'].nunique())  # print total no, of unique images
print('Number of classes in diff. categories = ', r_new_data['obj_class'].value_counts())

Number of unique images =  36
Number of classes in diff. categories =  buyer      38
date       36
vendor     36
invoice    36
po         33
order       1
Name: obj_class, dtype: int64


In [10]:
# We have to correct above :

# To find smallest width & height boxes in 'buyer' which should be 'po'
gb = r_new_data.groupby('obj_class')    
[gb.get_group('buyer') for x in gb.groups]

[       img_id  img_idx                                           i_path    x  \
 4     50a.jpg       24   /home/scar3crow/Downloads/8-6-new-scan/50a.jpg    5   
 9     51a.jpg       25   /home/scar3crow/Downloads/8-6-new-scan/51a.jpg    4   
 14    52a.jpg       26   /home/scar3crow/Downloads/8-6-new-scan/52a.jpg    1   
 19    53a.jpg       27   /home/scar3crow/Downloads/8-6-new-scan/53a.jpg    0   
 24    54a.jpg       28   /home/scar3crow/Downloads/8-6-new-scan/54a.jpg   31   
 29    55a.jpg       29   /home/scar3crow/Downloads/8-6-new-scan/55a.jpg    1   
 34    56a.jpg       30   /home/scar3crow/Downloads/8-6-new-scan/56a.jpg    1   
 39    59a.jpg       31   /home/scar3crow/Downloads/8-6-new-scan/59a.jpg    3   
 44    60a.jpg       32   /home/scar3crow/Downloads/8-6-new-scan/60a.jpg    0   
 49    61a.jpg       33   /home/scar3crow/Downloads/8-6-new-scan/61a.jpg    1   
 54    62a.jpg       34   /home/scar3crow/Downloads/8-6-new-scan/62a.jpg    4   
 59    63a.jpg       35   /h

In [11]:
# Correcting above wrong spelling & converting buyer to po of object classes and rechecking

id_1 = r_new_data.index[r_new_data['obj_class'] == 'order'] # Finding the index
id_2 = r_new_data.index[r_new_data['obj_class'] == 'date'] # to change 'date' to 'inv_date' to be consistent with old data

r_new_data.at[id_1, 'obj_class'] = 'po' # writing the correct spelling 
r_new_data.at[88, 'obj_class'] = 'po' # # 'buyer' to 'po'
r_new_data.at[163, 'obj_class'] = 'po' # # 'buyer' to 'po'
r_new_data.at[id_2, 'obj_class'] = 'inv_date' # # 'date' to 'inv_date'

print('Number of unique images = ', r_new_data['img_id'].nunique())  # print total no, of unique images
print('Number of unique classes = ', r_new_data['obj_class'].nunique())
print('Number of classes in diff. categories = ', r_new_data['obj_class'].value_counts())

# r_new_data.drop(r_new_data.columns[[0]], axis=1, inplace=True) # reduce unnecessary columns

Number of unique images =  36
Number of unique classes =  5
Number of classes in diff. categories =  buyer       36
inv_date    36
vendor      36
invoice     36
po          36
Name: obj_class, dtype: int64


In [12]:
## For each image, we have to find : (a) line_index = integer, (b) img_path = string, (c) boxes = shape [N, 4], 
## N is the ground truth count, elements in the second dimension are [x_min, y_min, x_max, y_max] (d) labels = shape
## [N]. class index. (e) img_width = int.  =f) img_height = int

def single_image_info(lines):
    
    ## lines will be a dataframe like, for i in range(num_images), lines = r_new_data[i*5:(i+1)*5]
    
    line_idx = lines.iat[0, 1]
    pic_path = lines.iat[0, 2]
    img_width = lines.iat[0, 8]
    img_height = lines.iat[0, 9]
    
    boxes = []
    labels = []
    for i in range(len(lines)):
        label, x_min, y_min, x_max, y_max = int(i), float(lines.iat[i,3]), float(lines.iat[i,4]), float(lines.iat[i,3]+lines.iat[i,5]), float(lines.iat[i,4]+lines.iat[i,6])
        boxes.append([x_min, y_min, x_max, y_max])
        labels.append(label)
        
    boxes = np.asarray(boxes, np.float32)
    labels = np.asarray(labels, np.int64)
    
    return line_idx, pic_path, boxes, labels, img_width, img_height  ## boxes are in format xmin, ymin, xmax, ymax


In [13]:
## Creating the complete data set :

all_image_line = []
for i in range(num_images):
    image_line = []
    limit_lower = i*5
    limit_upper = limit_lower+5
    lines = r_new_data[limit_lower:limit_upper]
    line_idx, pic_path, boxes, labels, img_width, img_height = single_image_info(lines)
    image_line.append(line_idx)
    image_line.append(pic_path)
    image_line.append(boxes)
    image_line.append(labels)
    image_line.append(img_width)
    image_line.append(img_height)
    all_image_line.append(image_line)
    
print(len(all_image_line))
print(all_image_line[1])   ##  boxes are in format xmin, ymin, xmax, ymax

36
[25, '/home/scar3crow/Downloads/8-6-new-scan/51a.jpg', array([[  5.,   0., 125.,  56.],
       [239.,   1., 279.,  20.],
       [328.,   1., 382.,  21.],
       [238.,  51., 302.,  74.],
       [  4.,  53., 156., 117.]], dtype=float32), array([0, 1, 2, 3, 4]), 416, 194]


In [14]:
## Train and Test split

data_train, data_val = train_test_split(all_image_line, train_size = 0.8 , shuffle = True)

num_all_bbox = len(all_image_line) * len(all_image_line[0][2])
num_bb_train = len(data_train) * len(data_train[0][2])
num_bb_val = len(data_val) * len(data_val[0][2])
print(num_all_bbox, num_bb_train, num_bb_val)

180 140 40


/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
## calculating anchors from true boundary boxes :

def iou_kmeans(box, clusters):
    """
    Calculates the Intersection over Union (IoU) between a box and k clusters.
    :param box: tuple or array, shifted to the origin (i. e. width and height)
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: numpy array of shape (k, 0) where k is the number of clusters
    """
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
        raise ValueError("Box has no area")

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou = intersection / (box_area + cluster_area - intersection)

    return iou

def kmeans(boxes, k, dist=np.median):
    """
    Calculates k-means clustering with the Intersection over Union (IoU) metric.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param k: number of clusters
    :param dist: distance function
    :return: numpy array of shape (k, 2)
    """
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    # the Forgy method will fail if the whole array contains the same rows
    clusters = boxes[np.random.choice(rows, k, replace=False)]


    while True:
        for row in range(rows):
            distances[row] = 1 - iou_kmeans(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

In [16]:
## Finding out anchors :
## Firstly, converting true boundary box width, height to width & height with respect to target image :
## finaly find anchors. Anchors here are in absolute size w.r.t. target image but not as % of target image or 
## as multiple of unit grids.

# num_all_bb = len(r_new_data) # if no. of bboxes varies for images, this formula should be used 

num_all_bb = len(all_image_line) * len(all_image_line[0][2])  ## from all image line data

b_box_wrt_target = np.zeros((num_all_bb,2))

for i in range(num_all_bb):
    
    image_w = r_new_data['img_wd'][i]
    image_h = r_new_data['img_ht'][i]

    x_ratio = target_w / image_w 
    y_ratio = target_h / image_h
    
    anchor_w = r_new_data['width'][i] * x_ratio
    anchor_h = r_new_data['height'][i] * y_ratio
    b_box_wrt_target[i, 0] = anchor_w
    b_box_wrt_target[i, 1] = anchor_h
    
anchors_wrt_target = kmeans(b_box_wrt_target, num_anchors)

print(anchors_wrt_target.shape)
print(anchors_wrt_target)     ## anchors wrt target image in abs. value and in format width, height


(3, 2)
[[129.23076923  37.92592593]
 [ 68.07692308  65.30612245]
 [189.80769231 160.36697248]]


In [17]:
## Pre-processing the original data to get y_true :

def process_box(ori_boxes, ori_img_width, ori_img_height, labels, target_size, class_num, anchors):
    '''
    Generate the y_true label, i.e. the ground truth feature_map.
    params:
        boxes: [N, 5] shape, float32 dtype. `x_min, y_min, x_max, y_mix, mixup_weight`.
        labels: [N] shape, int64 dtype.
        class_num: int64 num.
        anchors: [3,2] shape, float32 dtype.
    '''
    
    img_width = ori_img_width
    img_height = ori_img_height
    boxes = ori_boxes           ## boxes in format xmin, ymin, xmax, ymax
    
    x_ratio = target_size[1] / img_width
    y_ratio = target_size[0] / img_height
    
    boxes_wrt_target = np.zeros((5,4))
    box_centers_target = np.zeros((5,2))

    boxes_wrt_target[:,0] = boxes[:,0] * x_ratio  # xmin absolute value wrt target image
    boxes_wrt_target[:,1] = boxes[:,1] * y_ratio  # ymin absolute value wrt target image
    boxes_wrt_target[:,2] = boxes[:,2] * x_ratio  # xmax absolute value wrt target image
    boxes_wrt_target[:,3] = boxes[:,3] * y_ratio  # ymax absolute value wrt target image
    
    # In above, boxes_wrt_target shape is (5, 4), now this will be taken to (5. 5) by adding 1 at end
#    boxes_wrt_target = np.concatenate((boxes_wrt_target, np.full(shape=(boxes_wrt_target.shape[0], 1), fill_value=1., dtype=np.float32)), axis=-1)
    box_centers_target = (boxes_wrt_target[:, 0:2] + boxes_wrt_target[:, 2:4]) / 2  ## centers wrt target, abs values
    
    box_sizes = boxes[:, 2:4] - boxes[:, 0:2]  #xmax-xmin = width and ymax-ymin = height wrt original image
    box_sizes[:,0] = box_sizes[:,0] * x_ratio  # width w.r.t target image in absolute value
    box_sizes[:,1] = box_sizes[:,1] * y_ratio  # width w.r.t target image in absolute value
    
#    y_true_13 = np.zeros((target_size[1] // 32, target_size[0] // 32, 3, 6 + class_num), np.float32)
    y_true_13 = np.zeros((target_size[1] // 32, target_size[0] // 32, 3, 5 + class_num), np.float32)

#    y_true = [y_true_13]
    
    box_sizes = np.expand_dims(box_sizes, 1)
    mins = np.maximum(- box_sizes / 2, - anchors / 2)
    maxs = np.minimum(box_sizes / 2, anchors / 2)
    whs = maxs - mins

    iou = (whs[:, :, 0] * whs[:, :, 1]) / (
                box_sizes[:, :, 0] * box_sizes[:, :, 1] + anchors[:, 0] * anchors[:, 1] - whs[:, :, 0] * whs[:, :,
                                                                                                         1] + 1e-10)
    best_match_idx = np.argmax(iou, axis=1)

    anchor_mask = np.zeros((target_size[1] // 32, target_size[0] // 32, 3))

    cell_size = 32  ## = targetsize / no. of grid cells
    
    for i, idx in enumerate(best_match_idx):

        x = int(np.floor(box_centers_target[i, 0] / cell_size))
        y = int(np.floor(box_centers_target[i, 1] / cell_size))
        k = int(idx)
        c = int(labels[i])

#        print(x, y, k, c)

# Very Imp : Now preparing y_true: all values x_center, y_cemter, width & height are being taken to % of target image
        
        y_true_13[y, x, k, :2] = box_centers_target[i] / target_size[0] #since target_size[0] = target_size[1]
        y_true_13[y, x, k, 2:4] = box_sizes[i] / target_size[0]
        y_true_13[y, x, k, 4] = 1.
        y_true_13[y, x, k, 5 + c] = 1.
#        y_true[0][y, x, k, -1] = boxes_wrt_target[i, -1]
        anchor_mask[y, x, k] = 1

    return y_true_13, anchor_mask  ## all data are w.r.to target image in % of target image and NOT w,r,t, grid cells

In [18]:
## Single image-wise image/boundary box preprocessing:

def parse_data(line, class_num, target_size, anchors):   ## (mode, letterbox_resize):
    '''
    param:
        line: a line from the training/test txt file
        class_num: totol class nums.
        target_size: the size of image to be resized to. [width, height] format.
        anchors: anchors.
        mode: 'train' or 'val'. When set to 'train', data_augmentation will be applied.
        letterbox_resize: whether to use the letterbox resize, i.e., keep the original aspect ratio in the resized image.
    '''
    
    img_idx, pic_path, boxes, labels,img_width, img_height = line  # boxes in format xmin, ymin, xmax, ymax
    img = cv2.imread(pic_path)
    img_resized = cv2.resize(img,(target_size[0], target_size[1]))
    
    # expand the 2nd dimension, mix up weight default to 1.
    boxes = np.concatenate((boxes, np.full(shape=(boxes.shape[0], 1), fill_value=1., dtype=np.float32)), axis=-1)

    img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB).astype(np.float32)

    # the input of yolo_v3 should be in range 0~1, lets change to -0.5 to +0.5
    img_resized = (img_resized - 127.5)/ 255.

    y_true_13, anchor_mask = process_box(boxes, img_width, img_height, labels, target_size, class_num, anchors)

    return img_idx, img_resized, y_true_13, anchor_mask


In [19]:
## Making the data ready for entering into network :

anchors = anchors_wrt_target
image_index = []
image_resized = []
image_y_true = []
image_anchor_mask = []

for i in range(len(data_train)):

    line = data_train[i]
    
    img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)
    
    
    image_index.append(img_idx)
    image_resized.append(img_resized)
    image_y_true.append(y_true)
    image_anchor_mask.append(anchor_mask)
    
train_image_index = image_index
X_train = np.array(image_resized)
Y_train = np.array(image_y_true)
train_anchor_mask = np.array(image_anchor_mask)

image_index = []
image_resized = []
image_y_true = []
image_anchor_mask = []

for i in range(len(data_val)):
    line = data_val[i]
    
    img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)
    image_index.append(img_idx)
    image_resized.append(img_resized)
    image_y_true.append(y_true)
    image_anchor_mask.append(anchor_mask)
val_image_index = image_index
X_val = np.array(image_resized)
Y_val = np.array(image_y_true)
val_anchor_mask = np.array(image_anchor_mask)

image_index = []
image_resized = []
image_y_true = []


In [20]:
print(len(X_train), len(Y_train), len(train_image_index), len(X_val), len(Y_val), len(val_image_index))
print(train_image_index)
print(X_train.shape, Y_train.shape)
print(train_image_index[0], X_train[0].shape, Y_train[0].shape)
print(train_anchor_mask.shape, 'and', train_anchor_mask[0].shape)

28 28 28 8 8 8
[26, 13, 9, 4, 35, 15, 20, 6, 24, 33, 29, 23, 0, 30, 5, 22, 3, 1, 18, 10, 14, 34, 2, 7, 27, 28, 19, 11]
(28, 480, 480, 3) (28, 15, 15, 3, 10)
26 (480, 480, 3) (15, 15, 3, 10)
(28, 15, 15, 3) and (15, 15, 3)


In [42]:
reqd_index = int(r_new_data[r_new_data['img_idx'] == 18].index[0])
reqd_index

150

In [43]:
r_new_data[150:155]

,img_id,img_idx,i_path,x,y,width,height,obj_class,img_wd,img_ht
150,119a.jpg,18,/home/scar3crow/Downloads/8-6-new-scan/119a.jpg,56,6,135,55,vendor,416,146
151,119a.jpg,18,/home/scar3crow/Downloads/8-6-new-scan/119a.jpg,240,3,73,22,invoice,416,146
152,119a.jpg,18,/home/scar3crow/Downloads/8-6-new-scan/119a.jpg,333,1,50,21,inv_date,416,146
153,119a.jpg,18,/home/scar3crow/Downloads/8-6-new-scan/119a.jpg,240,36,63,24,po,416,146
154,119a.jpg,18,/home/scar3crow/Downloads/8-6-new-scan/119a.jpg,0,62,169,78,buyer,416,146


In [21]:
# https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/utils.py

def xywh_to_x1y1x2y2(box):
    xy = box[..., 0:2]
    wh = box[..., 2:4]

    x1y1 = xy - wh / 2
    x2y2 = xy + wh / 2

    y_box = K.concatenate([x1y1, x2y2], axis=-1)
    return y_box

In [22]:
# https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/utils.py

def broadcast_iou(box_a, box_b):
    """
    calculate iou between box_a and multiple box_b in a broadcast way
    inputs: box_a: a tensor full of boxes, eg. (B, N, 4), box is in x1y1x2y2
            box_b: another tensor full of boxes, eg. (B, M, 4)
    """

    # (B, N, 1, 4)
    box_a = tf.expand_dims(box_a, -2)
    # (B, 1, M, 4)
    box_b = tf.expand_dims(box_b, -3)
    # (B, N, M, 4)
    new_shape = tf.broadcast_dynamic_shape(tf.shape(box_a), tf.shape(box_b))

    # (B, N, M, 4)
    # (B, N, M, 4)
    box_a = tf.broadcast_to(box_a, new_shape)
    box_b = tf.broadcast_to(box_b, new_shape)

    # (B, N, M, 1)
    al, at, ar, ab = tf.split(box_a, 4, -1)
    bl, bt, br, bb = tf.split(box_b, 4, -1)

    # (B, N, M, 1)
    left = tf.math.maximum(al, bl)
    right = tf.math.minimum(ar, br)
    top = tf.math.maximum(at, bt)
    bot = tf.math.minimum(ab, bb)

    # (B, N, M, 1)
    iw = tf.clip_by_value(right - left, 0, 1)
    ih = tf.clip_by_value(bot - top, 0, 1)
    i = iw * ih

    # (B, N, M, 1)
    area_a = (ar - al) * (ab - at)
    area_b = (br - bl) * (bb - bt)
    union = area_a + area_b - i

    # (B, N, M)
    iou = tf.squeeze(i / (union + 1e-7), axis=-1)

    return iou


In [23]:
## https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/yolov3.py#L213

def calc_ignore_mask(ignore_thresh, true_box, pred_box):
    
        # YOLOv3:
        # "If the bounding box prior is not the best but does overlap a ground
        # truth object by more than some threshold we ignore the prediction,
        # following [17]. We use the threshold of .5."
        # calculate the iou for each pair of pred bbox and true bbox, then find the best among them

        # (None, 13, 13, 3, 4)
        
        true_box_reorganised = xywh_to_x1y1x2y2(true_box)  # reorganised to x1, y1, x2, y2
        pred_box_reorganised = xywh_to_x1y1x2y2(pred_box)
        
        true_box_shape = tf.shape(true_box_reorganised)  
        # (None, 13, 13, 3, 4)
        pred_box_shape = tf.shape(pred_box_reorganised)  
        # (None, 507, 4)
        true_box_reorganised = tf.reshape(true_box_reorganised, [true_box_shape[0], -1, 4])
        # sort true_box to have non-zero boxes rank first
        true_box_reorganised = tf.sort(true_box_reorganised, axis=1, direction="DESCENDING")
        # (None, 100, 4)
        # only use maximum 100 boxes per groundtruth to calcualte IOU, otherwise
        # GPU emory comsumption would explode for a matrix like (16, 52*52*3, 52*52*3, 4)
        true_box_reorganised = true_box_reorganised[:, 0:100, :]
        # (None, 507, 4)
        pred_box_reorganised = tf.reshape(pred_box_reorganised, [pred_box_shape[0], -1, 4])

        # https://github.com/dmlc/gluon-cv/blob/06bb7ec2044cdf3f433721be9362ab84b02c5a90/gluoncv/model_zoo/yolo/yolo_target.py#L198
        # (None, 507, 507)
        iou = broadcast_iou(pred_box_reorganised, true_box_reorganised)
        # (None, 507)
        best_iou = tf.reduce_max(iou, axis=-1)
        # (None, 13, 13, 3)
        best_iou = tf.reshape(best_iou, [pred_box_shape[0], pred_box_shape[1], pred_box_shape[2], pred_box_shape[3]])
        # ignore_mask = 1 => don't ignore
        # ignore_mask = 0 => should ignore
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        # (None, 13, 13, 3, 1)
        ignore_mask = tf.expand_dims(ignore_mask, axis=-1)
        
        return ignore_mask

In [24]:
def binary_cross_entropy(logits, labels):
    epsilon = 1e-7
    logits = tf.clip_by_value(logits, epsilon, 1 - epsilon)
    return -(labels * tf.math.log(logits) +
             (1 - labels) * tf.math.log(1 - logits))


In [25]:
# def yolo_my_loss(y_true, y_pred, anchors_wrt_target):
def yolo_my_loss(y_true, y_pred):
    
    '''Return yolo_loss
    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh ## w.r.t. target size
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss
    Returns
    -------
    loss: tensor, shape=(1,)
    '''
    grid_size = [15., 15.]
    ratio = 480./15.      ## ratio = 32
    ignore_thresh = 0.5
    Lambda_Coord = 5.0
    Lambda_no_obj = 0.5
    
    grid_x = np.arange(grid_size[1])
    grid_y = np.arange(grid_size[0])
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)
    grid_1st = np.dstack((grid_x, grid_y))
    grid_2nd = np.dstack((grid_1st, grid_x, grid_y))
    grid_3rd = np.dstack((grid_2nd, grid_x, grid_y)) 
    grid_final = np.reshape(grid_3rd, [1,15,15,3,2])
    
    loss = tf.zeros(1, dtype='float32')
    
    
#    m = y_true.shape[0]
#    m = np.expand_dims(m, axis=-1)
#    mf = K.cast(m, dtype='float32')

    
    obj_mask_y = y_true[..., 4:5]
    obj_mask = K.cast(obj_mask_y, dtype='float32')
    
#    true_box_wh = y_true[..., 2:4]
#    weight = 2 - true_box_wh[..., 0] * true_box_wh[..., 1]
#    weight = np.expand_dims(weight, axis=-1)
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
    pred_box_xy = K.sigmoid(y_pred[..., :2]) + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
                                                            # image = (x & y in no. of cells) / no. of cells
        
#    Lambda_Coord = K.cast(Lambda_Coord, dtype = 'float32')
    pred_box_xy_wrt_target_image = pred_box_xy / grid_size[0]
    true_box_xy_wrt_target_image = y_true[..., :2]
    
    pred_box_xy_wrt_ti = K.cast(pred_box_xy_wrt_target_image, dtype = 'float32')
    true_box_xy_wrt_ti = K.cast(true_box_xy_wrt_target_image, dtype = 'float32')
    
    xy_arr = K.cast((true_box_xy_wrt_ti - pred_box_xy_wrt_ti), dtype='float32')
    
    xy_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(xy_arr) * obj_mask), dtype = 'float32')
    
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord*tf.reduce_sum(tf.square(true_box_xy_wrt_ti - pred_box_xy_wrt_ti) *obj_mask))/m
#    wh_loss = tf.reduce_sum(tf.square(true_tw_th - pred_tw_th) * object_mask * box_loss_scale * mix_w) / N
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
    pred_box_wdht = K.exp(y_pred[..., 2:4]) * (np.reshape(anchors_wrt_target, [1,1,1,3,2])/480.)
    pred_box_wh = K.cast(pred_box_wdht, dtype = 'float32')
    true_box_wdht = y_true[..., 2:4]
    true_box_wh = K.cast(true_box_wdht, dtype = 'float32')
    
    wh_arr = K.cast((K.sqrt(true_box_wh) - K.sqrt(pred_box_wh)), dtype='float32')
    
    wh_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(wh_arr) * obj_mask), dtype = 'float32')
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

    pred_obj_mask = K.cast(K.sigmoid(y_pred[..., 4:5]), dtype = 'float32')  # shape = 28, 15, 15, 3, 1
       
    true_box_wrt_ti = K.concatenate([true_box_xy_wrt_ti, true_box_wh], axis = -1)  ## in x,y,w,h format
    pred_box_wrt_ti = K.concatenate([pred_box_xy_wrt_ti, pred_box_wh], axis = -1)  ## in x,y,w,h format
    
    
    
    
    true_box_wrt_ti = xywh_to_x1y1x2y2(true_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
    pred_box_wrt_ti = xywh_to_x1y1x2y2(pred_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
    
    ignore_mask = calc_ignore_mask(ignore_thresh, true_box_wrt_ti, pred_box_wrt_ti)
    
    ignore_mask = K.cast(ignore_mask, dtype = 'float32')                               

    obj_loss = K.cast(K.sum(K.binary_crossentropy(obj_mask, pred_obj_mask) * obj_mask), dtype = 'float32')
    
    no_obj_mask = 1. - obj_mask_y
    no_obj_mask = K.cast(no_obj_mask, dtype  = 'float32')
    
    noobj_loss = K.cast(Lambda_no_obj * K.sum(binary_cross_entropy(obj_mask, pred_obj_mask) * no_obj_mask * ignore_mask), dtype='float32')
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

    true_classes = K.cast(y_true[..., 5:10], dtype = 'float32')
    
    pred_classes = K.cast(y_pred[..., 5:10], dtype = 'float32')
    
    class_loss = K.cast(K.sum(binary_cross_entropy(true_classes, pred_classes) * obj_mask), dtype = 'float32')

    loss = xy_loss + wh_loss + obj_loss + noobj_loss + class_loss
    
    return loss

In [28]:
def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = InceptionResNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    new_last_layer = UpSampling2D(2)(last_layer)
    
    conv = Conv2D(512,(3,3) , padding='valid')(new_last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(256,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    another_up_layer = UpSampling2D(2)(lr)
    
    conv = Conv2D(256,(3,3) , padding='valid')(another_up_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(256,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(128,(3,3) , strides = (2, 2), padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(64,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(64,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(30,(1,1) , padding='same')(lr)
#    conv = Dropout(0.4)(conv)
#    bn = BatchNormalization()(conv)
#    lr = LeakyRelu(alpha=0.1)(bn)

    final = Reshape((grid_y_axis,grid_x_axis,num_anchors,info))(conv)

    model = Model(inp,final)
    features = model.output

    return model, features

input_size = (target_w, target_h, 3)
my_yolo_invoice_model, features_1 = yolo_model(input_size)


my_model_1, features_2 = yolo_model(input_size)
my_model_2, features_3 = yolo_model(input_size)

print(my_model_1.summary())


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1056 (Conv2D)            (None, 239, 239, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1051 (Batch (None, 239, 239, 32) 96          conv2d_1056[0][0]                
__________________________________________________________________________________________________
activation_1016 (Activation)    (None, 239, 239, 32) 0           batch_normalization_1051[0][0]   
____________________________________________________________________________________________

In [32]:
opt = SGD(learning_rate=0.001, momentum=0.9)

#opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
my_yolo_invoice_model.compile(optimizer= opt,loss=yolo_my_loss,metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [33]:
my_yolo_invoice_model.fit(X_train ,Y_train ,epochs= 5,batch_size = 4, validation_data=(X_val,Y_val))


Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 217s 8s/step - loss: -19092.4835 - accuracy: 0.1885 - val_loss: nan - val_accuracy: 0.9926
Epoch 2/5
28/28 [==============================] - 125s 4s/step - loss: nan - accuracy: 0.7013 - val_loss: nan - val_accuracy: 0.9926
Epoch 3/5
28/28 [==============================] - 130s 5s/step - loss: nan - accuracy: 0.9926 - val_loss: nan - val_accuracy: 0.9926
Epoch 4/5
28/28 [==============================] - 128s 5s/step - loss: nan - accuracy: 0.9926 - val_loss: nan - val_accuracy: 0.9926
Epoch 5/5
28/28 [==============================] - 124s 4s/step - loss: nan - accuracy: 0.9926 - val_loss: nan - val_accuracy: 0.9926


In [29]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
my_model_1.compile(optimizer= opt,loss=yolo_my_loss,metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [41]:
features = my_model_1(K.cast(X_train, dtype='float32'))

In [37]:
features

<tf.Tensor 'model_6_4/reshape_6/Reshape:0' shape=(28, 15, 15, 3, 10) dtype=float32>

In [50]:
my_model_1.fit(X_train ,Y_train ,epochs= 5,batch_size = 4, validation_data=(X_val,Y_val))


Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 918s 33s/step - loss: 9394.0702 - accuracy: 0.2917 - val_loss: 9478.5361 - val_accuracy: 0.0776
Epoch 2/5
28/28 [==============================] - 163s 6s/step - loss: 9098.0688 - accuracy: 0.3067 - val_loss: 9348.0894 - val_accuracy: 0.1091
Epoch 3/5
28/28 [==============================] - 130s 5s/step - loss: 8930.1405 - accuracy: 0.3236 - val_loss: 9225.4761 - val_accuracy: 0.1906
Epoch 4/5
28/28 [==============================] - 135s 5s/step - loss: 8704.7838 - accuracy: 0.3325 - val_loss: 9070.7363 - val_accuracy: 0.2630
Epoch 5/5
28/28 [==============================] - 133s 5s/step - loss: 8751.2394 - accuracy: 0.3468 - val_loss: 8890.9492 - val_accuracy: 0.2402


In [51]:
my_model_1.fit(X_train ,Y_train ,epochs= 20,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/20
28/28 [==============================] - 140s 5s/step - loss: 8511.7186 - accuracy: 0.3562 - val_loss: 8783.0176 - val_accuracy: 0.2919
Epoch 2/20
28/28 [==============================] - 135s 5s/step - loss: 8229.4413 - accuracy: 0.3592 - val_loss: 8653.1201 - val_accuracy: 0.3841
Epoch 3/20
28/28 [==============================] - 136s 5s/step - loss: 8071.1187 - accuracy: 0.3546 - val_loss: 8581.0859 - val_accuracy: 0.4254
Epoch 4/20
28/28 [==============================] - 137s 5s/step - loss: 8062.8636 - accuracy: 0.3758 - val_loss: 8510.5464 - val_accuracy: 0.4619
Epoch 5/20
28/28 [==============================] - 139s 5s/step - loss: 7805.0398 - accuracy: 0.3753 - val_loss: 8456.2837 - val_accuracy: 0.4756
Epoch 6/20
28/28 [==============================] - 133s 5s/step - loss: 7524.6754 - accuracy: 0.3767 - val_loss: 8387.5942 - val_accuracy: 0.4561
Epoch 7/20
28/28 [==============================] - 140s 5s/step - loss: 73

In [52]:
my_model_1.fit(X_train ,Y_train ,epochs= 30,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/30
28/28 [==============================] - 130s 5s/step - loss: 5156.8812 - accuracy: 0.3794 - val_loss: 6804.6787 - val_accuracy: 0.2307
Epoch 2/30
28/28 [==============================] - 131s 5s/step - loss: 4971.8205 - accuracy: 0.3723 - val_loss: 6516.6235 - val_accuracy: 0.2180
Epoch 3/30
28/28 [==============================] - 130s 5s/step - loss: 4866.8857 - accuracy: 0.3708 - val_loss: 6211.7239 - val_accuracy: 0.2174
Epoch 4/30
28/28 [==============================] - 134s 5s/step - loss: 4704.8507 - accuracy: 0.3657 - val_loss: 6072.5815 - val_accuracy: 0.2111
Epoch 5/30
28/28 [==============================] - 131s 5s/step - loss: 4513.3638 - accuracy: 0.3717 - val_loss: 5846.2275 - val_accuracy: 0.2196
Epoch 6/30
28/28 [==============================] - 131s 5s/step - loss: 4260.0818 - accuracy: 0.3604 - val_loss: 5787.7104 - val_accuracy: 0.2187
Epoch 7/30
28/28 [==============================] - 134s 5s/step - loss: 41

In [53]:
my_model_1.fit(X_train ,Y_train ,epochs= 30,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/30
28/28 [==============================] - 137s 5s/step - loss: 535.2450 - accuracy: 0.3273 - val_loss: 5181.5815 - val_accuracy: 0.0452
Epoch 2/30
28/28 [==============================] - 130s 5s/step - loss: 109.1204 - accuracy: 0.3214 - val_loss: 5280.4321 - val_accuracy: 0.0463
Epoch 3/30
28/28 [==============================] - 134s 5s/step - loss: -102.3944 - accuracy: 0.3137 - val_loss: 5433.0435 - val_accuracy: 0.0387
Epoch 4/30
28/28 [==============================] - 130s 5s/step - loss: -291.1480 - accuracy: 0.3239 - val_loss: 5434.8159 - val_accuracy: 0.0357
Epoch 5/30
28/28 [==============================] - 136s 5s/step - loss: -646.4450 - accuracy: 0.3190 - val_loss: 5580.3936 - val_accuracy: 0.0381
Epoch 6/30
28/28 [==============================] - 129s 5s/step - loss: -809.1694 - accuracy: 0.3106 - val_loss: 5782.7688 - val_accuracy: 0.0354
Epoch 7/30
28/28 [==============================] - 130s 5s/step - loss: -124

In [35]:
my_model_1.fit(X_train ,Y_train ,epochs= 5,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 382s 14s/step - loss: 8811.4082 - accuracy: 0.0901 - val_loss: 8984.5151 - val_accuracy: 0.0031
Epoch 2/5
28/28 [==============================] - 136s 5s/step - loss: 8491.1466 - accuracy: 0.0925 - val_loss: 9065.9468 - val_accuracy: 0.0233
Epoch 3/5
28/28 [==============================] - 143s 5s/step - loss: 8321.8136 - accuracy: 0.0874 - val_loss: 9070.8750 - val_accuracy: 0.0348
Epoch 4/5
28/28 [==============================] - 144s 5s/step - loss: 8033.6103 - accuracy: 0.0933 - val_loss: 9077.6299 - val_accuracy: 0.0248
Epoch 5/5
28/28 [==============================] - 143s 5s/step - loss: 7824.0460 - accuracy: 0.0918 - val_loss: 9047.2627 - val_accuracy: 0.0269


In [37]:
opt = SGD(learning_rate=0.0001, momentum=0.9)

#opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
my_yolo_invoice_model.compile(optimizer= opt,loss=yolo_my_loss,metrics=['accuracy'])

In [38]:
my_yolo_invoice_model.fit(X_train ,Y_train ,epochs= 5,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 572s 20s/step - loss: 1266.8813 - accuracy: 0.1751 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/5
28/28 [==============================] - 136s 5s/step - loss: -9979.0062 - accuracy: 0.3103 - val_loss: -6546343518535680.0000 - val_accuracy: 0.0000e+00
Epoch 3/5
28/28 [==============================] - 133s 5s/step - loss: -44420.4068 - accuracy: 0.3247 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/5
28/28 [==============================] - 129s 5s/step - loss: -261599.7868 - accuracy: 0.2849 - val_loss: -13014438504262129812504576.0000 - val_accuracy: 0.0000e+00
Epoch 5/5
28/28 [==============================] - 137s 5s/step - loss: nan - accuracy: 0.4024 - val_loss: nan - val_accuracy: 0.9926


In [86]:
grid_x = np.arange(5)
grid_y = np.arange(5)
print(grid_x, grid_y)

#grid_x, grid_y = np.meshgrid(grid_x, grid_y)
a = np.array(np.meshgrid(grid_x, grid_y))
b = np.array(np.meshgrid(grid_x, grid_y))
c = np.array(np.meshgrid(grid_x, grid_y))
d = np.concatenate((a,b,c), axis = 0)
#print(grid_x, grid_y)
print(a)
print(a.shape)
#b = 5 + a
#print(d)
print(d.shape)
#print(b[0,0,4])
#print(b[1,0,4])

[0 1 2 3 4] [0 1 2 3 4]
[[[0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]]

 [[0 0 0 0 0]
  [1 1 1 1 1]
  [2 2 2 2 2]
  [3 3 3 3 3]
  [4 4 4 4 4]]]
(2, 5, 5)
(6, 5, 5)


In [102]:
e = d.transpose(2, 1, 0)
f = np.reshape(e,[5,5,3,2])
print(e.shape)
print(d)
print('===============================================')
print(f[0,0],f[0,1],f[0,2])

(5, 5, 6)
[[[0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]]

 [[0 0 0 0 0]
  [1 1 1 1 1]
  [2 2 2 2 2]
  [3 3 3 3 3]
  [4 4 4 4 4]]

 [[0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]]

 [[0 0 0 0 0]
  [1 1 1 1 1]
  [2 2 2 2 2]
  [3 3 3 3 3]
  [4 4 4 4 4]]

 [[0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]]

 [[0 0 0 0 0]
  [1 1 1 1 1]
  [2 2 2 2 2]
  [3 3 3 3 3]
  [4 4 4 4 4]]]
[[0 0]
 [0 0]
 [0 0]] [[0 1]
 [0 1]
 [0 1]] [[0 2]
 [0 2]
 [0 2]]


In [5]:
Y = np.ones((5,30))
Y = np.reshape(Y,[5,5,3,2])
Y.shape

(5, 5, 3, 2)

In [105]:
Y+f

array([[[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[1., 3.],
         [1., 3.],
         [1., 3.]],

        [[1., 4.],
         [1., 4.],
         [1., 4.]],

        [[1., 5.],
         [1., 5.],
         [1., 5.]]],


       [[[2., 1.],
         [2., 1.],
         [2., 1.]],

        [[2., 2.],
         [2., 2.],
         [2., 2.]],

        [[2., 3.],
         [2., 3.],
         [2., 3.]],

        [[2., 4.],
         [2., 4.],
         [2., 4.]],

        [[2., 5.],
         [2., 5.],
         [2., 5.]]],


       [[[3., 1.],
         [3., 1.],
         [3., 1.]],

        [[3., 2.],
         [3., 2.],
         [3., 2.]],

        [[3., 3.],
         [3., 3.],
         [3., 3.]],

        [[3., 4.],
         [3., 4.],
         [3., 4.]],

        [[3., 5.],
         [3., 5.],
         [3., 5.]]],


       [[[4., 1.],
         [4., 1.],
         [4., 1.]],

        [[4., 2.],
         [4., 2.],
         [4.

In [6]:
g = np.arange(5)
h = np.arange(5)
g,h = np.meshgrid(g, h)
grid_1st = np.dstack((g, h))
grid_2nd = np.dstack((grid_1st, g, h))
grid_3rd = np.dstack((grid_2nd, g, h)) 
grid_final = np.reshape(grid_3rd, [5,5,3,2])
Y+grid_final

array([[[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[2., 1.],
         [2., 1.],
         [2., 1.]],

        [[3., 1.],
         [3., 1.],
         [3., 1.]],

        [[4., 1.],
         [4., 1.],
         [4., 1.]],

        [[5., 1.],
         [5., 1.],
         [5., 1.]]],


       [[[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[2., 2.],
         [2., 2.],
         [2., 2.]],

        [[3., 2.],
         [3., 2.],
         [3., 2.]],

        [[4., 2.],
         [4., 2.],
         [4., 2.]],

        [[5., 2.],
         [5., 2.],
         [5., 2.]]],


       [[[1., 3.],
         [1., 3.],
         [1., 3.]],

        [[2., 3.],
         [2., 3.],
         [2., 3.]],

        [[3., 3.],
         [3., 3.],
         [3., 3.]],

        [[4., 3.],
         [4., 3.],
         [4., 3.]],

        [[5., 3.],
         [5., 3.],
         [5., 3.]]],


       [[[1., 4.],
         [1., 4.],
         [1., 4.]],

        [[2., 4.],
         [2., 4.],
         [2.

In [ ]:
def yolo_my_loss(y_true, y_pred):
    
    '''Return yolo_loss
    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh ## w.r.t. target size
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss
    Returns
    -------
    loss: tensor, shape=(1,)
    '''
    grid_size = [15., 15.]
    ratio = 480./15.      ## ratio = 32
    ignore_thresh = 0.5
    Lambda_Coord = 5.0
    Lambda_no_obj = 0.5
    
    grid_x = np.arange(grid_size[1])
    grid_y = np.arange(grid_size[0])
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)
    grid_1st = np.dstack((grid_x, grid_y))
    grid_2nd = np.dstack((grid_1st, grid_x, grid_y))
    grid_3rd = np.dstack((grid_2nd, grid_x, grid_y)) 
    grid_final = np.reshape(grid_3rd, [1,15,15,3,2])
    
    loss = tf.zeros(1, dtype='float32')
    
    
#    m = y_true.shape[0]
#    m = np.expand_dims(m, axis=-1)
#    mf = K.cast(m, dtype='float32')

    
    obj_mask_y = y_true[..., 4:5]
    obj_mask = K.cast(obj_mask_y, dtype='float32')
    
#    true_box_wh = y_true[..., 2:4]
#    weight = 2 - true_box_wh[..., 0] * true_box_wh[..., 1]
#    weight = np.expand_dims(weight, axis=-1)
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
    pred_box_xy = K.sigmoid(y_pred[..., :2]) + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
                                                            # image = (x & y in no. of cells) / no. of cells
        
#    Lambda_Coord = K.cast(Lambda_Coord, dtype = 'float32')
    pred_box_xy_wrt_target_image = pred_box_xy / grid_size[0]
    true_box_xy_wrt_target_image = y_true[..., :2]
    
    pred_box_xy_wrt_ti = K.cast(pred_box_xy_wrt_target_image, dtype = 'float32')
    true_box_xy_wrt_ti = K.cast(true_box_xy_wrt_target_image, dtype = 'float32')
    
    xy_arr = K.cast((true_box_xy_wrt_ti - pred_box_xy_wrt_ti), dtype='float32')
    
    xy_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(xy_arr) * obj_mask), dtype = 'float32')
    
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord*tf.reduce_sum(tf.square(true_box_xy_wrt_ti - pred_box_xy_wrt_ti) *obj_mask))/m
#    wh_loss = tf.reduce_sum(tf.square(true_tw_th - pred_tw_th) * object_mask * box_loss_scale * mix_w) / N
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
    pred_box_wdht = K.exp(y_pred[..., 2:4]) * (np.reshape(anchors_wrt_target, [1,1,1,3,2])/480.)
    pred_box_wh = K.cast(pred_box_wdht, dtype = 'float32')
    true_box_wdht = y_true[..., 2:4]
    true_box_wh = K.cast(true_box_wdht, dtype = 'float32')
    
    wh_arr = K.cast((K.sqrt(true_box_wh) - K.sqrt(pred_box_wh)), dtype='float32')
    
    wh_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(wh_arr) * obj_mask), dtype = 'float32')
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

    pred_obj_mask = K.cast(K.sigmoid(y_pred[..., 4:5]), dtype = 'float32')  # shape = 28, 15, 15, 3, 1
       
    true_box_wrt_ti = K.concatenate([true_box_xy_wrt_ti, true_box_wh], axis = -1)  ## in x,y,w,h format
    pred_box_wrt_ti = K.concatenate([pred_box_xy_wrt_ti, pred_box_wh], axis = -1)  ## in x,y,w,h format
    
    
    
    
    true_box_wrt_ti = xywh_to_x1y1x2y2(true_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
    pred_box_wrt_ti = xywh_to_x1y1x2y2(pred_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
    
    ignore_mask = calc_ignore_mask(ignore_thresh, true_box_wrt_ti, pred_box_wrt_ti)
    
    ignore_mask = K.cast(ignore_mask, dtype = 'float32')                               

    obj_loss = K.cast(K.sum(K.binary_crossentropy(obj_mask, pred_obj_mask) * obj_mask), dtype = 'float32')
    
    no_obj_mask = 1. - obj_mask_y
    no_obj_mask = K.cast(no_obj_mask, dtype  = 'float32')
    
    noobj_loss = K.cast(Lambda_no_obj * K.sum(binary_cross_entropy(obj_mask, pred_obj_mask) * no_obj_mask * ignore_mask), dtype='float32')
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

    true_classes = K.cast(y_true[..., 5:10], dtype = 'float32')
    
    pred_classes = K.cast(y_pred[..., 5:10], dtype = 'float32')
    
    class_loss = K.cast(K.sum(binary_cross_entropy(true_classes, pred_classes) * obj_mask), dtype = 'float32')

    loss = xy_loss + wh_loss + obj_loss + noobj_loss + class_loss
    
    return loss

In [ ]:
true_box_reorganised = xywh_to_x1y1x2y2(true_box) 

In [ ]:
def calc_ignore_mask(ignore_thresh, true_box, pred_box):
    
        # YOLOv3:
        # "If the bounding box prior is not the best but does overlap a ground
        # truth object by more than some threshold we ignore the prediction,
        # following [17]. We use the threshold of .5."
        # calculate the iou for each pair of pred bbox and true bbox, then find the best among them

        # (None, 13, 13, 3, 4)
        
        true_box_reorganised = xywh_to_x1y1x2y2(true_box)  # reorganised to x1, y1, x2, y2
        pred_box_reorganised = xywh_to_x1y1x2y2(pred_box)
        
        true_box_shape = tf.shape(true_box_reorganised)  
        # (None, 13, 13, 3, 4)
        pred_box_shape = tf.shape(pred_box_reorganised)  
        # (None, 507, 4)
        true_box_reorganised = tf.reshape(true_box_reorganised, [true_box_shape[0], -1, 4])
        # sort true_box to have non-zero boxes rank first
        true_box_reorganised = tf.sort(true_box_reorganised, axis=1, direction="DESCENDING")
        # (None, 100, 4)
        # only use maximum 100 boxes per groundtruth to calcualte IOU, otherwise
        # GPU emory comsumption would explode for a matrix like (16, 52*52*3, 52*52*3, 4)
        true_box_reorganised = true_box_reorganised[:, 0:100, :]
        # (None, 507, 4)
        pred_box_reorganised = tf.reshape(pred_box_reorganised, [pred_box_shape[0], -1, 4])

        # https://github.com/dmlc/gluon-cv/blob/06bb7ec2044cdf3f433721be9362ab84b02c5a90/gluoncv/model_zoo/yolo/yolo_target.py#L198
        # (None, 507, 507)
        iou = broadcast_iou(pred_box_reorganised, true_box_reorganised)
        # (None, 507)
        best_iou = tf.reduce_max(iou, axis=-1)
        # (None, 13, 13, 3)
        best_iou = tf.reshape(best_iou, [pred_box_shape[0], pred_box_shape[1], pred_box_shape[2], pred_box_shape[3]])
        # ignore_mask = 1 => don't ignore
        # ignore_mask = 0 => should ignore
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        # (None, 13, 13, 3, 1)
        ignore_mask = tf.expand_dims(ignore_mask, axis=-1)
        
        return ignore_mask